In [1]:
import pandas as pd

In [2]:
import xgboost as xg

In [3]:
train = pd.read_csv("data/train.csv", parse_dates=["Dates"])
test = pd.read_csv("data/test.csv", parse_dates=["Dates"])

print(train.shape)
train.head()

(878049, 9)


,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [4]:
train["Dates-year"] = train["Dates"].dt.year
train["Dates-month"] = train["Dates"].dt.month
train["Dates-day"] = train["Dates"].dt.day
train["Dates-hour"] = train["Dates"].dt.hour
train["Dates-minute"] = train["Dates"].dt.minute
train["Dates-second"] = train["Dates"].dt.second

In [5]:
test["Dates-year"] = test["Dates"].dt.year
test["Dates-month"] = test["Dates"].dt.month
test["Dates-day"] = test["Dates"].dt.day
test["Dates-hour"] = test["Dates"].dt.hour
test["Dates-minute"] = test["Dates"].dt.minute
test["Dates-second"] = test["Dates"].dt.second

In [6]:
pd_district_list = train["PdDistrict"].unique()

pd_disrict_column_list = []

for pd_district in pd_district_list:
    pd_disrict_column = "PdDistrict_" + pd_district
    pd_disrict_column_list.append(pd_disrict_column)

    train[pd_disrict_column] = train["PdDistrict"] == pd_district

In [7]:
for pd_district in pd_district_list:
    pd_disrict_column = "PdDistrict_" + pd_district
    
    test[pd_disrict_column] = test["PdDistrict"] == pd_district

In [8]:
import numpy as np

train["Dates-minute-abs"] = np.abs(train["Dates-minute"] - 30)

In [9]:
test["Dates-minute-abs"] = np.abs(test["Dates-minute"] - 30)

In [10]:
train["CrossRoad"] = train["Address"].str.contains("/")

In [11]:
test["CrossRoad"] = test["Address"].str.contains("/")

In [12]:
feature_names = ["X", "Y", "Dates-hour", "Dates-minute-abs", "CrossRoad"]
feature_names = feature_names + pd_disrict_column_list

In [13]:
X_train = train[feature_names]

In [14]:
X_test = test[feature_names]

In [15]:
label_name = "Category"

y_train = train[label_name]

In [16]:
model = xg.XGBClassifier(n_estimators=45,
                         max_depth=6,
                         learning_rate=1.0,
                         max_delta_step=1,
                         nthread=4,
                         seed=37)
model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=1.0, max_delta_step=1,
       max_depth=6, min_child_weight=1, missing=None, n_estimators=45,
       n_jobs=1, nthread=4, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=37, silent=True,
       subsample=1)

In [ ]:
from sklearn.model_selection import cross_val_score

%time score = cross_val_score(model, X_train, y_train, cv=5, scoring="neg_log_loss").mean()
score = -1.0 * score

print("Score = {0:.5f}".format(score))